<a href="https://colab.research.google.com/github/DurgalakshmiU/durgaa_INFO5731_Spring2021/blob/main/In_class_exercise_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The ninth in-class-exercise (20 points in total, 4/16/2021)**

The purpose of the exercise is to practice different machine learning algorithms for text classification as well as the performance evaluation. In addition, you are requried to conduct *10 fold cross validation (https://scikit-learn.org/stable/modules/cross_validation.html)* in the training. 

The dataset can be download from here: https://github.com/unt-iialab/info5731_spring2021/blob/main/class_exercises/exercise09_datacollection.zip. The dataset contains two files train data and test data for sentiment analysis in IMDB review, it has two categories: 1 represents positive and 0 represents negative. You need to split the training data into training and validate data (80% for training and 20% for validation, https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6) and perform 10 fold cross validation while training the classifier. The final trained model was final evaluated on the test data. 

Algorithms:

(1) MultinominalNB

(2) SVM 

(3) KNN 

(4) Decision tree

(5) Random Forest

(6) XGBoost

Evaluation measurement:

(1) Accuracy

(2) Recall

(3) Precison 

(4) F-1 score

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_validate, StratifiedKFold
from xgboost import XGBClassifier

train= pd.read_csv(r'/content/stsa-train.txt',sep = 'delimiter=',header= None,names=['REVIEW'])
test= pd.read_csv(r'/content/stsa-test.txt',sep = 'delimiter=',header= None,names=['REVIEW'])

train[['SENTIMENT','REVIEW']] = train['REVIEW'].str.split(" ", 1, expand=True)
test[['SENTIMENT','REVIEW']] = test['REVIEW'].str.split(" ", 1, expand=True)
train.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if sys.path[0] == '':


,REVIEW,SENTIMENT
0,"a stirring , funny and finally transporting re...",1
1,apparently reassembled from the cutting-room f...,0
2,they presume their audience wo n't sit still f...,0
3,this is a visually stunning rumination on love...,1
4,jonathan parker 's bartleby should have been t...,1


In [ ]:
test.head()

,REVIEW,SENTIMENT
0,"no movement , no yuks , not much of anything .",0
1,"a gob of drivel so sickly sweet , even the eag...",0
2,"gangs of new york is an unapologetic mess , wh...",0
3,"we never really feel involved with the story ,...",0
4,this is one of polanski 's best films .,1


In [ ]:
import nltk
import re
import string
nltk.download('stopwords')
nltk.download('wordnet')
stopword=nltk.corpus.stopwords.words('english')
from nltk.stem import WordNetLemmatizer
wl= WordNetLemmatizer()

def ctext(texts):
  texts="".join([word.lower() for word in texts if word not in string.punctuation])
  texts= re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", texts)
  tokens = re.split('\W+',texts)
  texts = [wl.lemmatize(word) for word in tokens if word not in stopword]
  return texts

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
#Convert train data into numerical using TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer = ctext)
X_tfidf = tfidf_vect.fit_transform(train['REVIEW'])
print(X_tfidf.shape)

(6920, 13343)


In [ ]:
X_tfidf_df=pd.DataFrame(X_tfidf.toarray())
X_tfidf_df.columns=tfidf_vect.get_feature_names()
X_tfidf_df.head()

,,100minute,103minute,10course,10th,10thgrade,10year,10yearold,112minute,12,129minute,12th,12yearold,13th,14yearold,15th,15year,168minute,18yearold,1930s,1940s,1950s,1960s,1970s,1980s,19th,19thcentury,20car,20th,21st,22yearold,24andunders,26yearold,2day,30,34th,37minute,3d,3yearolds,40,...,yuen,yung,yvan,zaidan,zany,zap,zaza,zboys,zeal,zealand,zealously,zeitgeist,zelda,zellweger,zemeckis,zen,zero,zerodimensional,zeus,zhang,zhao,zhuangzhuang,zigzag,zing,zinger,zingerfilled,zip,zipper,zippy,zishe,ziyi,zoe,zombie,zombieland,zone,zoning,zoom,zwick,zzzzzzzzz,élan
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.400584,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.048154,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.029784,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.049596,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.048782,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#Convert train data into numerical using TF-IDF
X_test_tfidf = tfidf_vect.transform(test['REVIEW'])
print(X_test_tfidf.shape)

(1821, 13343)


In [ ]:
#Algorithms
MNB = MultinomialNB()
SVM = LinearSVC()
KNN = KNeighborsClassifier(n_neighbors=5,n_jobs=-1)
DT= DecisionTreeClassifier()
RF= RandomForestClassifier()
XGB= XGBClassifier()

In [ ]:
#spliting train data
x_train, x_test, y_train, y_test = train_test_split(X_tfidf_df, train['SENTIMENT'].values,
                                                test_size=0.2, random_state=42)
model_mnb = MNB.fit(x_train,y_train)

In [ ]:
#Accuracy using MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
y_pred_mnb = model_mnb.predict(x_test)
print('Accuracy: %s' % accuracy_score(y_pred_mnb,y_test))
print(classification_report(y_test,y_pred_mnb))


Accuracy: 0.7955202312138728
              precision    recall  f1-score   support

           0       0.85      0.70      0.77       671
           1       0.76      0.88      0.82       713

    accuracy                           0.80      1384
   macro avg       0.80      0.79      0.79      1384
weighted avg       0.80      0.80      0.79      1384



In [ ]:
from sklearn.model_selection import cross_val_score
scores_m = cross_val_score(MNB, x_test, y_test, cv=10)
print("BY USING MNB:",scores_m.mean())


BY USING MNB: 0.7247054530288813


In [ ]:
#Accuracy using SVM
model_svm = SVM.fit(x_train,y_train)
y_pred_svm = model_svm.predict(x_test)
print('Accuracy: %s' % accuracy_score(y_pred_svm,y_test))
print(classification_report(y_test,y_pred_svm))

Accuracy: 0.791907514450867
              precision    recall  f1-score   support

           0       0.81      0.75      0.78       671
           1       0.78      0.83      0.80       713

    accuracy                           0.79      1384
   macro avg       0.79      0.79      0.79      1384
weighted avg       0.79      0.79      0.79      1384



In [ ]:
from sklearn.model_selection import cross_val_score
scores_s = cross_val_score(SVM, x_test, y_test, cv=10)
print("BY USING SVM:",scores_s.mean())

BY USING SVM: 0.7348034615785632


In [ ]:
#KNNeighbors
model_knn = KNN.fit(x_train,y_train)
y_pred_knn = model_knn.predict(x_test)
print('Accuracy: %s' % accuracy_score(y_pred_knn,y_test))
print(classification_report(y_test,y_pred_knn))

Accuracy: 0.7398843930635838
              precision    recall  f1-score   support

           0       0.74      0.70      0.72       671
           1       0.74      0.77      0.75       713

    accuracy                           0.74      1384
   macro avg       0.74      0.74      0.74      1384
weighted avg       0.74      0.74      0.74      1384



In [ ]:
from sklearn.model_selection import cross_val_score
scores_k = cross_val_score(KNN, x_test, y_test, cv=10)
print("BY USING KNN:",scores_k.mean())

BY USING KNN: 0.6675737670732979


In [ ]:
#Decision tree
model_dt = DT.fit(x_train,y_train)
y_pred_dt = model_dt.predict(x_test)
print('Accuracy: %s' % accuracy_score(y_pred_dt,y_test))
print(classification_report(y_test,y_pred_dt))

Accuracy: 0.6582369942196532
              precision    recall  f1-score   support

           0       0.66      0.61      0.63       671
           1       0.66      0.70      0.68       713

    accuracy                           0.66      1384
   macro avg       0.66      0.66      0.66      1384
weighted avg       0.66      0.66      0.66      1384



In [ ]:
scores_d = cross_val_score(DT, x_test, y_test, cv=10)
print("BY USING DT:",scores_d.mean())

BY USING DT: 0.6141434678344281


In [ ]:
#Random Forest
model_rf = RF.fit(x_train,y_train)
y_pred_rf = model_rf.predict(x_test)
print('Accuracy: %s' % accuracy_score(y_pred_rf,y_test))
print(classification_report(y_test,y_pred_rf))

Accuracy: 0.7485549132947977
              precision    recall  f1-score   support

           0       0.79      0.66      0.72       671
           1       0.72      0.83      0.77       713

    accuracy                           0.75      1384
   macro avg       0.75      0.75      0.75      1384
weighted avg       0.75      0.75      0.75      1384



In [ ]:
scores_r = cross_val_score(RF, x_test, y_test, cv=10)
print("BY USING RF:",scores_r.mean())


BY USING RF: 0.6719111667187988


In [ ]:
#XGBoost
model_xgb = XGB.fit(x_train,y_train)
y_pred_xgb = model_xgb.predict(x_test)
print('Accuracy: %s' % accuracy_score(y_pred_xgb,y_test))
print(classification_report(y_test,y_pred_xgb))

Accuracy 0.6445086705202312
              precision    recall  f1-score   support

           0       0.75      0.40      0.52       671
           1       0.61      0.88      0.72       713

    accuracy                           0.64      1384
   macro avg       0.68      0.64      0.62      1384
weighted avg       0.68      0.64      0.62      1384



In [ ]:
scores_x = cross_val_score(XGB, x_test, y_test, cv=10)
print("BY USING XGB:",scores_x.mean())


BY USING XGB: 0.6184704410384736


In [ ]:
print("Accuracy with MNB:",scores_m.mean())
print("Accuracy with SVM:",scores_s.mean())
print("Accuracy with KNN:",scores_k.mean())
print("Accuracy with Decision trees(DT):",scores_d.mean())
print("Accuracy with Random Forest(RF):",scores_r.mean())
print("Accuracy with XGBoost(XGB):",scores_x.mean())

Accuracy with MNB: 0.7247054530288813
Accuracy with SVM: 0.7348034615785632
Accuracy with KNN: 0.6675737670732979
Accuracy with Decision trees(DT): 0.6141434678344281
Accuracy with Random Forest(RF): 0.6719111667187988
Accuracy with XGBoost(XGB): 0.6184704410384736


In [ ]:
#MultinomialNB and linearSVM algorithms got the best accuracy and precision values.


In [ ]:
#Prediction on test data

#MNB
predict_mnb = model_mnb.predict(X_test_tfidf)
print('Final trained model(MNB) with high accuracy evaluated on the test data: %s' % accuracy_score(predict_mnb,test['SENTIMENT']))

Final trained model(MNB) with high accuracy evaluated on the test data: 0.7946183415705657
